In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd gdrive/My Drive/Voice_Cloning/gantts

/content/gdrive/My Drive/Voice_Cloning/gantts


In [3]:
!pip install -e ".[train]"

Obtaining file:///content/gdrive/My%20Drive/Voice_Cloning/gantts
     |████████████████████████████████| 1.7MB 5.7MB/s 
     |████████████████████████████████| 419kB 23.1MB/s 
     |████████████████████████████████| 368kB 28.8MB/s 
  Created wheel for nnmnkwii: filename=nnmnkwii-0.0.20-cp36-cp36m-linux_x86_64.whl size=1880484 sha256=72ae69295345f8244b41372ee8d2df6b72d1bc0f8c5197015ec4b6c4e700f2a8
  Stored in directory: /root/.cache/pip/wheels/c8/b2/6a/76aedd175231e475da518b6a63fcc6ecee567e57f29cbc1b05
  Created wheel for pysptk: filename=pysptk-0.1.18-cp36-cp36m-linux_x86_64.whl size=949517 sha256=ec5e03ca460778629faaeeff5219651b96608632ab6db44f1d19d3a84d03a6be
  Stored in directory: /root/.cache/pip/wheels/c7/96/d2/a163240019c59504402fab713af259026af81a99dea943404a
  Created wheel for bandmat: filename=bandmat-0.7-cp36-cp36m-linux_x86_64.whl size=897981 sha256=817fc279b4d44591d470e9ba8a8919faff2dc5b861049629749e47bffb1b5c3a
  Stored in directory: /root/.cache/pip/wheels/54/1e/28/429ff

In [4]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [5]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 43kB/s 
     |████████████████████████████████| 3.8MB 46.8MB/s 
     |████████████████████████████████| 512kB 56.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=8803f025fb51c62306983376a124b4f9ce6d50eff834210906e3be371ed0e081
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


In [6]:

%pylab inline
rcParams["figure.figsize"] = (16,5)

import sys
sys.path.insert(0, "..")

Populating the interactive namespace from numpy and matplotlib


In [9]:

import torch

from scipy.io import wavfile
import pysptk
from pysptk.synthesis import Synthesizer, MLSADF
import pyworld
from os.path import join, basename

from nnmnkwii import preprocessing as P
from nnmnkwii.paramgen import unit_variance_mlpg_matrix

import gantts
from hparams import vc as hp

import librosa
import librosa.display
import IPython
from IPython.display import Audio

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [8]:
!pip install pyworld

     |████████████████████████████████| 81kB 3.5MB/s 
  Created wheel for pyworld: filename=pyworld-0.2.8-cp36-cp36m-linux_x86_64.whl size=613104 sha256=599402c41ee4cbfba2fe6546eddf4485839815658ddb1958760a2f7f7c30d262
  Stored in directory: /root/.cache/pip/wheels/c3/58/e5/a7e39ab92c56825f976970b97066753c68406c7fb0d80d4a4a
Successfully built pyworld


In [0]:

# data_dir = "./data/cmu_arctic/"
clb_wav_dir = "./data/cmu_arctic/cmu_us_clb_arctic/wav/"
slt_wav_dir = "./data/cmu_arctic/cmu_us_slt_arctic/wav/"

# data_mean = np.load(join(data_dir, "data_mean.npy"))
# data_var = np.load(join(data_dir, "data_var.npy"))
# data_std = np.sqrt(data_var)

In [13]:

if hp.generator_params["in_dim"] is None:
    hp.generator_params["in_dim"] = data_mean.shape[-1]
if hp.generator_params["out_dim"] is None:
    hp.generator_params["out_dim"] = data_mean.shape[-1]


NameError: ignored

In [0]:

fs = 16000
hop_length = int(fs * (hp.frame_period * 0.001))
fftlen = pyworld.get_cheaptrick_fft_size(fs)
static_dim = hp.order

In [15]:
model_in2out = getattr(gantts.models, hp.generator)(**hp.generator_params)
print(model_in2out)

checkpoint_path = "../checkpoints/vc/baseline/checkpoint_epoch200_Generator.pth"
print("Load checkpoint from: {}".format(checkpoint_path))
checkpoint = torch.load(checkpoint_path)
model_in2out.load_state_dict(checkpoint["state_dict"])

TypeError: ignored

In [16]:

model_gan = getattr(gantts.models, hp.generator)(**hp.generator_params)
print(model_gan)

checkpoint_path = "../checkpoints/vc/gan/checkpoint_epoch200_Generator.pth"
print("Load checkpoint from: {}".format(checkpoint_path))
checkpoint = torch.load(checkpoint_path)
model_gan.load_state_dict(checkpoint["state_dict"])

TypeError: ignored

In [0]:
from evaluation_vc import get_wav_files
from evaluation_vc import test_vc_from_path

src_test_files = get_wav_files(data_dir, clb_wav_dir, test=True)
tgt_test_files = get_wav_files(data_dir, slt_wav_dir, test=True)

for src_path, tgt_path in zip(src_test_files, tgt_test_files):
    fs, src_waveform = wavfile.read(src_path)
    fs, tgt_waveform = wavfile.read(tgt_path)
    
    in2out_waveform,_,_ = test_vc_from_path(model_in2out, src_path, data_mean, data_std, diffvc=True)
    gan_waveform,_,_ = test_vc_from_path(model_gan, src_path, data_mean, data_std, diffvc=True)
    
    print(basename(src_path), ": source, target, baseline, GAN")
    for x in [src_waveform, tgt_waveform, in2out_waveform, gan_waveform]:
        IPython.display.display(Audio(x, rate=fs))

In [0]:
# Pick your favorite sample
name = "arctic_a0001.wav"

src_path = join(clb_wav_dir, name)
tgt_path = join(slt_wav_dir, name)

In [0]:
def compute_static_features(path):
    fs, x = wavfile.read(path)
    x = x.astype(np.float64)
    f0, timeaxis = pyworld.dio(x, fs, frame_period=hp.frame_period)
    f0 = pyworld.stonemask(x, f0, timeaxis, fs)
    spectrogram = pyworld.cheaptrick(x, f0, timeaxis, fs)
    aperiodicity = pyworld.d4c(x, f0, timeaxis, fs)
    alpha = pysptk.util.mcepalpha(fs)
    mc = pysptk.sp2mc(spectrogram, order=hp.order, alpha=alpha)
    c0, mc = mc[:, 0], mc[:, 1:]
    return mc
    
def vis_gv(src_path, tgt_path):    
    tgt_mgc = compute_static_features(tgt_path)
    waveform_in2out, _, mgc_in2out = test_vc_from_path(
        model_in2out, src_path, data_mean, data_std, diffvc=True)
    waveform_gan, _, mgc_gan = test_vc_from_path(
        model_gan, src_path, data_mean, data_std, diffvc=True)
    
    figure(figsize=(16,6))
    plot(np.var(tgt_mgc, axis=0), "--", linewidth=2, label="Target: global variances")
    plot(np.var(mgc_in2out, axis=0), linewidth=2, label="In2Out: global variances")
    plot(np.var(mgc_gan, axis=0), linewidth=2, label="GAN: global variances")
    legend(prop={"size": 18})
    yscale("log")
    xlim(0, static_dim)
    xlabel("Dimention", fontsize=16)
        
    fs, src_waveform = wavfile.read(src_path)
    fs, tgt_waveform = wavfile.read(tgt_path)
    
    for x, name in [(src_waveform, "Source waveform"),
                   (tgt_waveform, "Target waveform"),
                   (waveform_in2out, "In2Out"),
                   (waveform_gan, "GAN")]:
        print(name)
        IPython.display.display(Audio(x, rate=fs))

In [0]:
vis_gv(src_path, tgt_path)
